In [151]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

In [ ]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged

In [181]:
def hod_rd_sub1(meanhls,meanc):
    for i in range(my,shape[1]-lmd,lmd-2):
        ver_rd(i,meanhls,meanc)

In [182]:
def hod_rd_sub2(mean1hls,mean1c):
    for i in range(my+lmd,0, -(lmd-2)):
        ver_rd(i,mean1hls,mean1c)

In [183]:
def hor_rd(meanhls,meanc):
    hod_rd_sub1(meanhls,meanc)
    hod_rd_sub2(meanhls,meanc)

In [215]:
def ver_rd(x,meanhls,meanc):
    lm1 = lim1
    lm2 = lim2
    lmc = limc
    for i in range(mx,lmd+1,-(lmd-10)):
        lm1 += lm1*0.01
        lm2 += lm2*0.01
        lmc += limc*0.02
        lowhls = meanhls - [0,lm1,lm2]
        highhls = meanhls + [180,lm1,lm2]
        lowc = meanc - lmc
        highc = meanc + lmc
        newscr[i-lmd:i, x:x+lmd] =  cv2.inRange(scr[i-lmd:i, x:x+lmd],lowc,highc)#cv2.inRange(hls[i-lmd:i, x:x+lmd],lowhls,highhls)#+
        
        reshls = [0]+[np.mean(hls[i-lmd:i, x:x+lmd,j]) for j in range(1,3)]
        reshls = np.array(reshls)
        
        resc = [np.mean(scr[i-lmd:i, x:x+lmd,j]) for j in range(3)]
        resc = np.mean(resc)
        
        if(abs(reshls[1]-meanhls[1])<=15 and abs(reshls[2]-meanhls[2])<=15):
            meanhls = (4*reshls+6*meanhls)/10
        
        
        if resc <= 0.8*np.mean(highc) and resc >= 0.8*np.mean(lowc):
            meanc = (5*resc+5*meanc)/10
      

In [225]:
import numpy as np
import pandas as pd
import time
import cv2
import threading
# lim = 4*std
lmd = 40
lim1 = lim2 = 10
limc = 20
kernel = np.ones((3,3),np.uint8)

shape = [480,680]#scr.shape
scr2 = 0
mx = shape[0] - 220
my = shape[1]//2
print("running")
kkk = 300*8*2
mask_prev1 = np.zeros((shape[0]-220,shape[1]),np.uint8)
mask_prev2 = np.zeros((shape[0]-220,shape[1]),np.uint8)
mean_prev_hls = 0
mean_prev_c = 0
vrec = cv2.VideoCapture('v4.mp4',0)

last = None

while(kkk):
    ret, scr = vrec.read()
    kkk-=1

while True:
    start_time = time.time()
    blank = cv2.imread('blank.png', 0)
    blank = cv2.resize(blank,(680,260))
    scr = cv2.imread("c2.jpg",1)
#     if not ret:
#         print("not running")
#         break
#     ret, scr = vrec.read()
#     if not ret:
#             break
    scr = cv2.resize(scr, (680,480)) 
    scr = scr[220:480,:]
    scr1 = scr
    scr2 = scr
    newscr = np.zeros((shape[0]-220,shape[1]),np.uint8)
    scr = cv2.bilateralFilter(scr,7,12,12)

###   HLS
    hls = cv2.cvtColor(scr, cv2.COLOR_BGR2HLS)
    meanhls = [0]+[np.mean(hls[440-220:480-220,200:480, i]) for i in range(1,3)]
    std = [np.std(hls[440-220:480-220,200:480, i]) for i in range(1,3)]
    lim1 = (lim1 + 16*std[0])/5  #(1:4(4))/5
    lim2 = (lim2 + 16*std[1])/5
#     print(lim1,lim2)
    meanhls = np.array(meanhls)
    meanhls = 0.3*meanhls + mean_prev_hls*0.7
    mean_prev_hls = meanhls
#     print(mean)

#### RGB

    meanc = [np.mean(scr[440-220:480-220,200:480, i]) for i in range(3)]
    meanc = np.array(meanc)
    meanc = 0.2*meanc + mean_prev_c*0.8
    mean_prev_c = meanc

####
    hor_rd(meanhls,meanc)
    
#### gray
#     band =  cv2.bitwise_and(scr,scr,mask = newscr)
    
    gray = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
    mean1 = np.mean(gray[440-220:480-220,200:480])
    mean1 = np.array(mean1)
    low1 = mean1 - 10
    high1 = mean1 + 10
    
#     mask1 = cv2.inRange(gray,low1,high1)
#     band1 =  cv2.bitwise_and(scr,scr,mask = mask1)
#     cv2.imshow('mask1',mask1)
    cv2.imshow('newscr',newscr)
    mask3 = newscr #+ mask1
    mask4 = mask_prev1 + mask3 
    mask_prev1 = mask3 
    
    canny = autocanny(gray,low1,high1)
    cv2.imshow("canny", canny)


#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 3)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)
    cv2.imshow('band3',mask4)
    dist_transform = cv2.distanceTransform(mask4,cv2.DIST_L2,5)
    ret, mask4 = cv2.threshold(dist_transform,0.07*dist_transform.max(),255,0)
    mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 3)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)

    cv2.imshow('band4',mask4)
    mask4 = np.array(mask4)
    
    
    mask4 = np.uint8(mask4)
    im2, contours, hierarchy = cv2.findContours(mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
#     im3, newcontours, newhierarchy = cv2.findContours(mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     scr1 = cv2.drawContours(scr1, newcontours, -1, (0,255,255), 3)
#     cv2.imshow('scr1',scr1)
    ncont=len(contours)
    points = []
    if not last:
        last = contours
#     set_bottom_boundary = set([[260,x] for x in range(200,480)])
    for i in range(ncont):
#         print(len(contours[i]),len(last[i]))
        if(True):#len(contours[i])>100):
            epsilon = 0.003*cv2.arcLength(contours[i], True)
#             if set(list(contours[i][0])).intersection(set_bottom_boundary):
#                 contours[i] = cv2.approxPolyDP(contours[i], epsilon, True)
#                 print(i)
#             else:
            contours[i] = cv2.approxPolyDP(contours[i], epsilon, True)
            
            
            
            
#     for i in range(min(ncont,len(last))):
#         print(len(contours[i]),len(last[i]))
#         if(True):#len(contours[i])>100):
#             epsilon = 0.003*cv2.arcLength(contours[i], True)
#             contours[i] = cv2.approxPolyDP(contours[i], epsilon, True)
#             for k in range(len(contours[i])):
#                 [point.append([i,j] for i incontours[i][k][0][0]:contours[i][k+1][0][0], contours[i][k][0][1]:contours[i][k+1][0][1]]
                
#             print(len(contours[i]))
    last = contours
    blank = cv2.drawContours(blank, contours, -1, 0, 3)
    scr = cv2.drawContours(scr, contours, -1, 0, 3)
        

    
#     cv2.line(scr2,(0,480),(340,240),(255,255,0),2)
#     cv2.line(scr2,(680,480),(340,240),(255,255,0),2)

    cv2.imshow('band4',mask4)
    cv2.imshow('scr2',scr)
    cv2.imshow('blank',blank)



#     cv2.line(scr2,(0, 480),(340, int(220)),(0, 255, 255), 1)
#     cv2.line(scr2,(680, 480),(340, int(220)),(0, 255, 255), 1)
    
#     cv2.line(scr2,(0,int(220)),(680,int(220)),(0,0,255),2)#216  220
    
#     for ind in range(0,260+20,20):
#         xx = (ind)*2
#         cv2.putText(scr2,str(xx),(10,480-ind), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,0),1,cv2.LINE_AA)
#         cv2.line(scr2,(0,480-ind),(680,480-ind),(0,255,255),1)

    
#     cv2.imshow('scr2',scr2)
#     time.sleep(0.2)
    print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running
FPS:  3.329394021178301
FPS:  12.782368071385905
FPS:  12.20573051560672
FPS:  12.97509729070897
FPS:  12.747172059154261
FPS:  13.103432763080614
FPS:  13.396736977935634
FPS:  13.692243909286482
FPS:  13.063559546390339
FPS:  10.07739861463832
FPS:  10.790372207415334
FPS:  13.009869910730348
FPS:  12.821363722511265
FPS:  13.24222935747955
FPS:  13.030199663861369
FPS:  12.42672054941441
FPS:  12.57277834065743
FPS:  12.831562008602702
FPS:  12.29172112651291
FPS:  10.52533526057978
FPS:  10.583151451229943
FPS:  12.92890235594751
FPS:  13.512794987032652
FPS:  13.337946474000203
FPS:  13.081445903377725
FPS:  13.02955201346973
FPS:  13.068810369539477
FPS:  11.618956862832006
FPS:  12.588740586891731
FPS:  11.107914278752952
FPS:  12.281679252962586
FPS:  12.888933958988257
FPS:  13.008498021263662
FPS:  12.878130258035199
FPS:  12.970523112699924
FPS:  13.335444466700368
FPS:  13.268453169930815
FPS:  13.139391509197534
FPS:  13.035059094822094
FPS:  13.159963227565521
FPS

FPS:  13.059085432111067
FPS:  13.227529179504808
FPS:  13.172610242736589
FPS:  12.989080548759716
FPS:  13.339219041197834
FPS:  13.138527175734643
FPS:  12.619571856242384
FPS:  13.298743464461573
FPS:  13.018511391147806
FPS:  13.189303413750597
FPS:  13.301020812654398
FPS:  13.284969498096402
FPS:  13.337225460361676
FPS:  12.835842271969153
FPS:  13.184079664543871
FPS:  13.191543430810746
FPS:  12.78380956735092
FPS:  13.459632053038787
FPS:  13.287957750271664
FPS:  13.419410983666875
FPS:  13.240808028512712
FPS:  13.240431845444789
FPS:  12.339781289257754
FPS:  13.349748238304699
FPS:  13.156825902701753
FPS:  13.278954983362935
FPS:  12.850629157232627
FPS:  13.35774113210912
FPS:  13.28867344675728
FPS:  13.453199473971196
FPS:  12.722503306276465
FPS:  13.393999003665998
FPS:  12.547390337953255
FPS:  13.072557722036603
FPS:  12.874019324976366
FPS:  13.064536124219346
FPS:  13.221899982346859
FPS:  13.213694116016269
FPS:  12.67781814664579
FPS:  12.735599097580897
FPS:

In [81]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

vrec = cv2.VideoCapture('v2.mp4',0)
shape = [480,680]

mx = shape[0]
my = shape[1]//2
print("running")
kkk = 300*6*6
while(kkk):
    kkk-=1
    ret, scr = vrec.read()

ret, scr1 = vrec.read()
scr1 = cv2.resize(scr1, (680,480)) 
scr1 = cv2.cvtColor(scr1, cv2.COLOR_BGR2GRAY)
ret, scr2 = vrec.read()
scr2 = cv2.resize(scr2, (680,480)) 
scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
while True:
    start_time = time.time()
    scr = scr1
    scr1 = scr2
    ret, scr2 = vrec.read()
#     ret, scr3 = vrec.read()
#     scr = np.mean(scr1,scr2,scr3)
    if not ret:
            break
    scr2 = cv2.resize(scr2, (680,480)) 
    scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
    
    d0 = abs(scr1-scr)
    d1 = abs(scr2-scr)
    d2=d0+d1
    
    cv2.imshow('d0',d0)
    cv2.imshow('d1',d1)
    cv2.imshow('d2',d2)
#     cv2.imshow('d3',d3)
#     cv2.imshow('d4',d4)

    cv2.imshow('scr1',scr)
    print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running


error: /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
